<a href="https://colab.research.google.com/github/SY-256/llms-from-scratch/blob/main/notebooks/ch04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPTモデルを1から実装

In [ ]:
# GPTモデルの設定
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # 語彙のサイズ
    "context_length": 1024, # コンテキストの長さ
    "emb_dim": 768,         # 埋め込み次元数
    "n_heads": 12,          # Attentionヘッドの数
    "n_layers": 12,         # 層の数
    "drop_rate": 0.1,       # ドロップアウト率
    "qkv_bias": False       # クエリ、キー、バリューの計算にバイアスを使用するか
}

In [ ]:
# GPTプレイスホルダモデルアーキテクチャ
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):

    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # TransformerBlockにはプレースホルダを使う
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        # LayerNormにもプレースホルダを使う(正規化層)
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

class DummyTransformerBlock(nn.Module):
    # 後ほど本物のTransformerBlockに置き換える
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        # 何もせずに入力を返すだけ
        return x

class DummyLayerNorm(nn.Module):
    # 後ほど本物のLAyerNormに置き換える
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()

    def forward(self, x):
        return x

In [ ]:
# GPTモデル用のトークナイザーを使用して
# 2つのテキスト入力からなるバッチをトークン化する
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

In [ ]:
# DummyGPTModelインスタンスを生成して
# トークン化したバッチbatchを入力として渡す
# モデルの出力は一般的にロジットと言う
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape: ", logits.shape)
print(logits)